In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font color="red" size="5">ch.11 데이터프레임과 시리즈</font></b>


** 주요함수 **
- columns, index : 열과 행 이름 바꾸기
- loc, iloc : 특정 데이터 조회하기
- merge() : 공통열 기준 병합
- concat() : 데이터 이어붙이기
- sort_index, sort_values : 행/열 이름 또는 값 기준으로 정렬
- mean(), median(), min(), max() : 평균, 중앙값, 최댓값, 최솟값 등
- groupby() : 기준 열로 데이터를 그룹화해서 요약 통계 내기
- melt(), pivot_table() : 와이드포맷↔롱포맷 변환
- apply(), applymap(), map() : 데이터프레임 전체나 각 요소에 함수 적용


# 1절 판다스 패키지
- 판다스(Pandas) : 데이터 분석을 위해 테이블 형식(엑셀) 데이터를 다루기 좋게 만들어주는 넘파이 기반 파이썬 라이브러리(엑셀을 파이썬으로 조작하는 기술)
    - 장점 : 파일 io, 부분 데이터 추출, 크기변경, 데이터 분할 및 병합, 정렬, 결측치 처리, 데이터 분할, 피벗과 언피벗(와이드포맷, 롱포맷)에 용이
    - [Pandas API reference] https://pandas.pydata.org/docs/reference/index.html#api
    - [Pandas docs] https://pandas.pydata.org/docs/
    
- 데이터 프레임(Data Frame) : 엑셀 시트처럼 행(Row) 과 열(Column) 로 이루어진 2차원 구조
- 시리즈(Series) : 데이터프레임의 한 열(Column) 을 의미(1차원 구조)
- 데이터프레임과 시리즈 요약 : 시리즈는 1열짜리 데이터, 데이터프레임은 여러 열짜리 표 전체다!
- DataFrame 만들기 : 딕셔너리, 리스트, csv 파일, sklearn, seaborn 등 다양한 방법으로 생성


In [5]:
import pandas as pd
pd.__version__

'1.5.3'

In [8]:
data = pd.read_csv('data/ch09_member1.csv', encoding='utf-8') #encoding='utf-8' 기본값으로 설정되어 있음.
display(data)
type(data)

,홍길동,20,kildong@hong.com,서울시 관악구
0,김길동,40,kimdong@hong.com,서울시 영등포구
1,신길동,30,sindong@hong.com,서울시 동작구


pandas.core.frame.DataFrame

In [12]:
data = pd.read_csv('data/ch09_member1.csv', 
                   header=None,
                   names=['name','age', 'email', 'address'])
data.head(2) # 최초 상위 2줄

,name,age,email,address
0,홍길동,20,kildong@hong.com,서울시 관악구
1,김길동,40,kimdong@hong.com,서울시 영등포구


In [15]:
data = pd.read_json('data/ch09_member1.json', encoding='utf-8')
data

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


In [17]:
iris = pd.read_hdf('data/ch09_iris.hdf5', key='iris')
iris.head() # 상위 5개

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# 2절. 데이터 프레임 만들기
## 2.1 딕셔너리를 이용해서 데이터프레임 만들기(리스트를 이용해서도 가능함.)

In [18]:
#딕셔너리를 데이터프레임으로
d = {'kor':[100,90], 'mat':[95,99]} 
df = pd.DataFrame(data=d)
df

,kor,mat
0,100,95
1,90,99


In [19]:
df.to_dict() #데이터 프레임을 딕셔너리로 변환

{'kor': {0: 100, 1: 90}, 'mat': {0: 95, 1: 99}}

In [20]:
df.to_numpy()

array([[100,  95],
       [ 90,  99]], dtype=int64)

In [22]:
# 딕셔너리 리스트를 데이터프레임으로
d = [{'kor':100, 'mat':95}, {'kor':90., 'mat':99}]
df = pd.DataFrame(data=d)
df

,kor,mat
0,100.0,95
1,90.0,99


In [23]:
df.dtypes #kor이 실수(float)로 입력됨. {'kor':90., 'mat':99}에서 90에 .을 찍었으므로

kor    float64
mat      int64
dtype: object

In [24]:
df.info() # ★

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kor     2 non-null      float64
 1   mat     2 non-null      int64  
dtypes: float64(1), int64(1)
memory usage: 160.0 bytes


In [37]:
d = [{'kor':100, 'mat':100}, {'kor':90, 'mat':91}, {'kor':93, 'eng':100}]
df = pd.DataFrame(data=d) 
# 결측치(NaN)는 실수형(float 64)으로 나타나며 정수형 변환이 불가능함(결측치대체를 한 후 정수형변환이 가능함.)
df

,kor,mat,eng
0,100,100.0,NaN
1,90,91.0,NaN
2,93,NaN,100.0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kor     3 non-null      int64  
 1   mat     2 non-null      float64
 2   eng     1 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 200.0 bytes


## 2.2 리스트를 이용한 데이터프레임 만들기

In [39]:
title = ['책1','책2', '책3']
price = [15000, 18000, 10000]
df = pd.DataFrame(data={'title':title, 'price':price})
# 딕셔너리 형태로 데이터를 구성하고, 이를 이용해 데이터프레임을 생성
df

,title,price
0,책1,15000
1,책2,18000
2,책3,10000


In [40]:
import numpy as np
np.c_[title, price]

array([['책1', '15000'],
       ['책2', '18000'],
       ['책3', '10000']], dtype='<U11')

In [48]:
df = pd.DataFrame(np.c_[title, price], columns=['책이름','가격'])
df

,책이름,가격
0,책1,15000
1,책2,18000
2,책3,10000


In [45]:
l = [['책1', 15000],
     ['책2', 18000],
     ['책3', 10000]]
df = pd.DataFrame(l, columns=['책이름','가격'])
df

,책이름,가격
0,책1,15000
1,책2,18000
2,책3,10000


[소상공인시장진흥공단_상가(상권)정보] https://www.data.go.kr/data/15083033/fileData.do
- 에서 데이터 다운로드, 데이터가 엄청나게 많다. 이런 데이터는 txt로 작업하거나 엑셀로 작업이 매우 비효율적이다.(용량 매우 큼)

## 2.3 read_csv

In [50]:
%ls c:\AI_x\01_download\shareData\상가정보_20250331 #폴더 내용물 확인

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 c:\AI_x\01_download\shareData\상가정보_20250331 디렉터리

2025-04-28  오후 03:51    <DIR>          .
2025-04-28  오후 03:51    <DIR>          ..
2025-04-27  오후 11:07             1,011 [필독]파일열람방법.txt
2025-04-27  오후 11:07        61,227,944 소상공인시장진흥공단_상가(상권)정보_강원_202503.csv
2025-04-27  오후 11:07       356,572,082 소상공인시장진흥공단_상가(상권)정보_경기_202503.csv
2025-04-27  오후 11:07        93,472,559 소상공인시장진흥공단_상가(상권)정보_경남_202503.csv
2025-04-27  오후 11:07        76,425,275 소상공인시장진흥공단_상가(상권)정보_경북_202503.csv
2025-04-27  오후 11:07        40,070,868 소상공인시장진흥공단_상가(상권)정보_광주_202503.csv
2025-04-27  오후 11:07        61,715,005 소상공인시장진흥공단_상가(상권)정보_대구_202503.csv
2025-04-27  오후 11:07        41,491,092 소상공인시장진흥공단_상가(상권)정보_대전_202503.csv
2025-04-27  오후 11:07        84,550,600 소상공인시장진흥공단_상가(상권)정보_부산_202503.csv
2025-04-27  오후 11:07       294,549,483 소상공인시장진흥공단_상가(상권)정보_서울_202503.csv
2025-04-27  오후 11:07         9,303,432 소상공인시장진흥공단_상가(상권)정보_세종_202503.csv
2025-04-27  오후 11:07        28,457,4

In [52]:
df = pd.read_csv(r'C:/AI_x/01_download/shareData/상가정보_20250331/소상공인시장진흥공단_상가(상권)정보_세종_202503.csv',
                sep=',',
                low_memory=False, #데이터 용량이 클 경우 기입
                nrows=30 # 처음 30행만 가져오라는 명령어
                )
df.shape

(30, 39)

In [53]:
df #그냥 무지성으로 열면 이모양남.

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220700019175,세종수이비인후과의원,NaN,Q1,보건의료,Q102,의원,Q10201,내과/소아과 의원,Q86201,...,3611011600200050002000001,행정중심복합도시,세종특별자치시 세종특별자치시 해밀3로 43,NaN,30092,NaN,NaN,NaN,127.264487,36.528585
1,MA010120220800005576,우보설렁탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,...,4473025023100230051035586,NaN,세종특별자치시 세종특별자치시 조치원읍 문화로 24,339885.0,30025,NaN,NaN,NaN,127.301068,36.603770
2,MA010120220800006182,메이빈,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,3611010300106640011000001,강남프라자,세종특별자치시 세종특별자치시 시청대로 127,339008.0,30151,NaN,3.0,NaN,127.286081,36.479674
3,MA010120220800008970,명품동태탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,...,3611035024103040008000001,행복마을,세종특별자치시 세종특별자치시 장군면 월현윗길 29,339914.0,30055,NaN,1.0,NaN,127.223073,36.501346
4,MA010120220800024017,컴119,NaN,S2,수리·개인,S201,컴퓨터 수리,S20101,컴퓨터/노트북/프린터 수리업,S95110,...,3611010600102180000000001,골든타원,세종특별자치시 세종특별자치시 노을3로 99,339014.0,30130,NaN,2.0,NaN,127.253843,36.479828
5,MA010120220800047182,포지션,NaN,G2,소매,G216,안경·정밀기기 소매,G21603,사진기/기타 광학기기 소매업,G47829,...,3611034022102850004000001,NaN,세종특별자치시 세종특별자치시 금남면 발산1길 67,339833.0,30083,NaN,NaN,NaN,127.283364,36.456746
6,MA010120220800046344,이화공인중개사사무소,NaN,L1,부동산,L102,부동산 서비스,L10203,부동산 중개/대리업,L68221,...,3611010700106840001000001,나릿재마을2단지,세종특별자치시 세종특별자치시 어울누리로 67,339003.0,30128,NaN,NaN,NaN,127.265373,36.487283
7,MA010120220800046670,김가네칼국수,NaN,I2,음식,I201,한식,I20105,국수/칼국수,I56112,...,4473025029102660002000001,NaN,세종특별자치시 세종특별자치시 조치원읍 큰말2길 3,339807.0,30030,NaN,NaN,NaN,127.289301,36.595365
8,MA010120220800048962,굿모닝보청기,NaN,G2,소매,G215,의약·화장품 소매,G21502,의료기기 소매업,G47812,...,4473025021100010014037459,NaN,세종특별자치시 세종특별자치시 조치원읍 조치원7길 6,339883.0,30024,NaN,1.0,NaN,127.300613,36.601726
9,MA010120220800042909,청년피자,조치원점,I2,음식,I210,기타 간이,I21003,피자,I56192,...,3611025028102670008000001,주건축물제,세종특별자치시 세종특별자치시 조치원읍 행복10길 9,339887.0,30021,NaN,2.0,NaN,127.291133,36.604147


In [58]:
# 판다스 디스플레이 옵션
pd.options.display.max_columns = 39 #기본값은 20으로 되어있음. 39열까지 display 가능
#max_columns	화면에 표시할 최대 열 수
pd.options.display.max_rows
#max_rows	화면에 표시할 최대 행 수(기본값 60)
df.shape

(30, 39)

In [59]:
df.head(2)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220700019175,세종수이비인후과의원,NaN,Q1,보건의료,Q102,의원,Q10201,내과/소아과 의원,Q86201,일반의원,36,세종특별자치시,36110,세종특별자치시,36110525,해밀동,3611011600,해밀동,3611011600200050002,2,산,5,2.0,세종특별자치시 세종특별자치시 해밀동 산5-2,361103352828,세종특별자치시 세종특별자치시 해밀3로,43,NaN,3611011600200050002000001,행정중심복합도시,세종특별자치시 세종특별자치시 해밀3로 43,NaN,30092,NaN,NaN,NaN,127.264487,36.528585
1,MA010120220800005576,우보설렁탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,한식 일반 음식점업,36,세종특별자치시,36110,세종특별자치시,36110250,조치원읍,3611025023,조치원읍,3611025023100230051,1,대지,23,51.0,세종특별자치시 세종특별자치시 조치원읍 평리 23-51,361103258043,세종특별자치시 세종특별자치시 조치원읍 문화로,24,NaN,4473025023100230051035586,NaN,세종특별자치시 세종특별자치시 조치원읍 문화로 24,339885.0,30025,NaN,NaN,NaN,127.301068,36.603770


In [60]:
df.head(1).T

,0
상가업소번호,MA010120220700019175
상호명,세종수이비인후과의원
지점명,NaN
상권업종대분류코드,Q1
상권업종대분류명,보건의료
상권업종중분류코드,Q102
상권업종중분류명,의원
상권업종소분류코드,Q10201
상권업종소분류명,내과/소아과 의원
표준산업분류코드,Q86201


In [61]:
# (1) CSV 파일 불러오기(기본값)
# encording=utf-8, sep=',', csv파일의 첫번째 줄을 header로 인식, #이 있는 줄도 데이터로 인식
member = pd.read_csv('data/ch11_member.csv')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30
5,#홍길뜽,24,non@not.com,없는구,2000-01-01


In [62]:
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     6 non-null      object
 1   Age      6 non-null      int64 
 2   Email    6 non-null      object
 3   Address  6 non-null      object
 4   Birth    6 non-null      object
dtypes: int64(1), object(4)
memory usage: 368.0+ bytes


In [67]:
# 형변환 Age열:int64 -> int16
member['Age'] = member['Age'].astype(np.int16) #'int' = np.int32와 동일함.
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     6 non-null      object
 1   Age      6 non-null      int16 
 2   Email    6 non-null      object
 3   Address  6 non-null      object
 4   Birth    6 non-null      object
dtypes: int16(1), object(4)
memory usage: 332.0+ bytes


In [68]:
# 형변환 Birth열 : object(문자) -> datetime64
member['Birth'].astype('datetime64')

0   1999-01-01
1   1999-12-01
2   1999-11-01
3   1999-05-01
4   1999-01-30
5   2000-01-01
Name: Birth, dtype: datetime64[ns]

In [69]:
member['Birth_as'] = member['Birth'].astype('datetime64')
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Name      6 non-null      object        
 1   Age       6 non-null      int16         
 2   Email     6 non-null      object        
 3   Address   6 non-null      object        
 4   Birth     6 non-null      object        
 5   Birth_as  6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(4)
memory usage: 380.0+ bytes


In [70]:
member['Birth_to'] = pd.to_datetime(member['Birth']) # astype()함수보다 안전한 형변환
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Name      6 non-null      object        
 1   Age       6 non-null      int16         
 2   Email     6 non-null      object        
 3   Address   6 non-null      object        
 4   Birth     6 non-null      object        
 5   Birth_as  6 non-null      datetime64[ns]
 6   Birth_to  6 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int16(1), object(4)
memory usage: 428.0+ bytes


In [72]:
# (2) 특정행을 제외하고 csv파일읽기
member = pd.read_csv('data/ch11_membercp949.csv',
                    encoding='cp949',
                    skiprows=[1, 6]) # 1, 6번째 행은 제외하고 읽어오기
member

,Name,Age,Email,Address,Birth
0,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
1,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
2,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
3,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [77]:
# (3) 주석(#)제외, datetime형 필드 지정하여 읽기
member = pd.read_csv('data/ch11_membertab.csv',
                    sep='\t',
                    comment='#',
                    parse_dates=['Birth']) # datetime형으로 읽어올 필드 지정
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [79]:
# (4) sep='/', 상위5행만 읽어오기
member = pd.read_csv('data/ch11_membersep.csv',
                    sep='|',
                    nrows=5)
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 2.4 패키지에서 가져오기 : iris 데이터
- 3가지 방법이 있음.
    - iris 가져오기 방법 1 : sklearn패키지(머신러닝 패키지)에서 가져오기
    - iris 가져오기 방법 2 : statsmodels (R 데이터)
    - iris 가져오기 방법 3 : seabron(시각화 패키지)

### iris 가져오기 방법 1 : sklearn패키지(머신러닝 패키지)에서 가져오기

In [85]:
from sklearn import datasets
# sklearn : 머신러닝을 위한 함수(전처리 함수, 머신러닝 함수, 성능평가를 위한 함수, 학습데이터셋)
iris = datasets.load_iris()
iris.keys() # 딕셔너리 형태

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [86]:
print(iris.DESCR) #iris 데이터 셋에 대한 설명

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [87]:
iris.data # 독립변수
iris['data'][:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [88]:
iris.feature_names #독립변수의 열이름
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [94]:
# 데이터 프레임에서 열이름
columns = [col[:-5].replace(' ','_') for col in iris.feature_names]+['species']
columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [97]:
iris.target
iris['target'] # 머신러닝시 종속변수

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [99]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [101]:
iris['target_names'][iris['target']]
iris.target_names[iris.target]

array(['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolo

In [103]:
# 독립변수
data = iris.data
# 종속변수
target = iris.target_names[iris.target]
data.shape, target.shape # 독립변수와 종속변수의 차원

((150, 4), (150,))

In [104]:
# 독립변수와 종속변수를 stack한 후 데이터프레임으로 
pd.DataFrame(data = np.hstack( (data, target.reshape(-1,1) ) ),
            columns=columns)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### iris 가져오는 방법 2 : statsmodels (R 데이터)

In [2]:
# R언어에 있는 유명한 데이터셋을 가져오는 함수
from statsmodels.datasets import get_rdataset
iris_dataset = get_rdataset('iris',
                           package='datasets', # R의 datasets 패키지에서 가져오고,
                           cache=True) # 한번 다운로드한 데이터셋을 PC에 저장
iris_dataset

<class 'statsmodels.datasets.utils.Dataset'>

In [4]:
iris = iris_dataset.data
iris.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [5]:
iris['Sepal.Length'] #iris에 엑세스 하는 방법
# iris.Sepal.Length 열이름에 .이나 (스페이스)가 있으면 접근불가

0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: Sepal.Length, Length: 150, dtype: float64

In [9]:
# 따라서 보다 편한 엑세스를 위해서 컬럼을 바꾸어야함.
iris.columns = [col.lower().replace('.','_') for col in iris.columns]
iris.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa


### iris 가져오기 방법 3 : seabron(시각화 패키지)

In [10]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa


In [11]:
#데이터 프레임을 csv파일로
iris.to_csv('data/ch11_iris.csv',
           # sep=',', encoding='utf-8', #기본값
           index=False) #행이름을 제외하고 파일 출력

In [12]:
import pandas as pd
load_iris = pd.read_csv('data/ch11_iris.csv')
load_iris.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa


In [13]:
# 데이터 프레임을 압축파일(.zip, .gz, .bz2)로 변환하는 방법
iris.to_csv('data/ch11_iris.gz',
           index=False,
           compression='infer')

In [15]:
iris.to_csv('data/ch11_iris.bz2',
           index=False,
           compression='infer')

In [14]:
iris.to_csv('data/ch11_iris.zip',
           index=False,
           compression='infer') #infer = 확장자 파일명에 따라 알아서 압축하도록하는 명령어

In [16]:
load_iris = pd.read_csv('data/ch11_iris.gz', 
                        compression='infer')
load_iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


# 3절. 이름(열이름, 행이름) 지정하기
## 3.1 열이름 지정하기

In [18]:
member = pd.read_csv('data/ch11_member.csv',
                    comment='#',
                    parse_dates=['Birth'])
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [21]:
# ↑ 열 이름을 2차원으로 변경하고 싶은 경우
member.columns = ['이름', '나이', '메일', '주소', '생년월일']
member.columns.name = None
member

,이름,나이,메일,주소,생년월일
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 3.2 행이름 지정

In [22]:
member.index

RangeIndex(start=0, stop=5, step=1)

In [24]:
member.index = range(11, 16)
member

,이름,나이,메일,주소,생년월일
11,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
12,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
13,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
14,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
15,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [25]:
member.index = ['동', '서', '남', '북', '중']
member

,이름,나이,메일,주소,생년월일
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [33]:
#이런식으로 추가할 수 있음
print(member.index.name)

방향


In [31]:
member.loc['남','이름'] # loc를 이용해서 행이름과 열이름으로 데이터 부분조회

'홍길남'

In [36]:
# 특정열을 index로 setting
# member를 수정하려면 (1) 할당, (2) inplace 매개변수에 True (기본값은 False)
member1 = member.set_index('이름') # 이름열이 index로 setting
member1

,나이,메일,주소,생년월일
이름,,,,
홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
홍길서,30,seo@hong.com,서울시 중구,1999-12-01
홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [40]:
member.set_index('이름', inplace=True)  #이름열을 index로
#inplace=True	원본 객체를 직접 수정	반환값 없음 (None 반환)
#inplace=False	수정된 복사본을 반환	원본은 그대로, 새 객체 반환

In [42]:
member.head(1)

,나이,메일,주소,생년월일
이름,,,,
홍길동,20,hong@hong.com,서울시 강동구,1999-01-01


In [44]:
member.reset_index(inplace=True) #index를 컬럼(열)로

In [45]:
member

,이름,나이,메일,주소,생년월일
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [47]:
member.loc[1, '생년월일']

Timestamp('1999-12-01 00:00:00')

In [49]:
# 시리즈(날짜형).dt : datetime열의 날짜 및 시간에 관련된 정보만 추출
member['생년월일'].dt.year

0    1999
1    1999
2    1999
3    1999
4    1999
Name: 생년월일, dtype: int64

In [50]:
member['생년월일'].dt.weekday # 0:월, 1:화, ... 5:토, 6:일

0    4
1    2
2    0
3    5
4    5
Name: 생년월일, dtype: int64

## 3.3 레벨 이름 지정하기

In [51]:
member

,이름,나이,메일,주소,생년월일
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [54]:
member.columns = [['기본정보','기본정보','기본정보','추가정보','추가정보'],
                  ['이름','나이','메일','주소','생년월일']]
member.columns.names = ['대분류','소분류']
member

대분류 기본정보                        추가정보           
소분류   이름  나이             메일       주소       생년월일
0    홍길동  20  hong@hong.com  서울시 강동구 1999-01-01
1    홍길서  30   seo@hong.com   서울시 중구 1999-12-01
2    홍길남  21   nam@hong.com  서울시 강동구 1999-11-01
3    홍길북  22   buk@hong.com  서울시 관악구 1999-05-01
4    홍길중  23   mid@hong.com   부산시 중구 1999-01-30

In [56]:
member.index = [['좌우','좌우','상하','상하','상하'],
               ['동','서','남','북','중']]
member.index.names = ['레벨1','레벨2']
member

대분류     기본정보                        추가정보           
소분류       이름  나이             메일       주소       생년월일
레벨1 레벨2                                            
좌우  동    홍길동  20  hong@hong.com  서울시 강동구 1999-01-01
    서    홍길서  30   seo@hong.com   서울시 중구 1999-12-01
상하  남    홍길남  21   nam@hong.com  서울시 강동구 1999-11-01
    북    홍길북  22   buk@hong.com  서울시 관악구 1999-05-01
    중    홍길중  23   mid@hong.com   부산시 중구 1999-01-30

In [57]:
member.loc[('좌우','동'),'기본정보']

소분류
이름              홍길동
나이               20
메일    hong@hong.com
Name: (좌우, 동), dtype: object

# 4절. 부분데이터 조회
- member['열이름'] 특정열
- member[조건]
- member.loc[행이름, 열이름:열이름]
- member.iloc[행번호, 열번호]

In [58]:
member = pd.read_csv('data/ch11_member.csv', comment='#', parse_dates=['Birth'])
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 4.1 열 조회하기

In [59]:
member['Name']
member.Name

0    홍길동
1    홍길서
2    홍길남
3    홍길북
4    홍길중
Name: Name, dtype: object

In [60]:
member[ ['Name','Email'] ]

,Name,Email
0,홍길동,hong@hong.com
1,홍길서,seo@hong.com
2,홍길남,nam@hong.com
3,홍길북,buk@hong.com
4,홍길중,mid@hong.com


In [61]:
# member[0] 행을 조회할 경우 loc, iloc. []안에는 열이름과 조건만 사용 가능

## 4.2 loc을 이용한 조회
- df.loc[행이름, 열이름] : 행이름과 열이름으로 조회, "이름"을 기준으로 행과 열을 조회하는 명령어
    * 행이름과 열이름 자리에 list ex.['Name', 'Email']
    * 행이름과 열이름 자리에 슬라이싱 from:to : from부터 to까지(from, to포함)
    * ,열이름 생략시 모든 열

In [128]:
#하나 조회	df.loc['홍길동', '나이']	홍길동의 나이만 조회
#여러 열 조회	df.loc['홍길동', ['나이', '이메일']]	홍길동의 나이와 이메일 조회
#여러 행 조회	df.loc[['홍길동', '홍길서']]	홍길동과 홍길서 전체 열 조회
#슬라이싱 조회	df.loc['홍길동':'홍길서']	홍길동부터 홍길서까지 (포함!)
#모든 열 조회	df.loc['홍길동', :]	홍길동의 모든 열 데이터 조회
#모든 행 조회	df.loc[:, '이메일']	모든 사람의 이메일만 조회

In [63]:
# member 0행부터 2행
member.loc[0:2]

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01


In [64]:
member.index = ['동','서','남','북','중']
member

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [65]:
# member 동행부터 남행
member.loc['동':'남']

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01


In [66]:
# member 동행부터 남행, 'Name', 'Age', 'Email'
member.loc['동':'남', 'Name':'Email']

,Name,Age,Email
동,홍길동,20,hong@hong.com
서,홍길서,30,seo@hong.com
남,홍길남,21,nam@hong.com


In [68]:
# member 동행과 남행, 'Name', Email', 'Address'
member.loc[['동','남'], ['Name','Email','Address'] ]

,Name,Email,Address
동,홍길동,hong@hong.com,서울시 강동구
남,홍길남,nam@hong.com,서울시 강동구


In [71]:
# loc을 이용한 특정 열 조회
member.loc[:,['Name','Address']] # member['Name','Address'] 동일

,Name,Address
동,홍길동,서울시 강동구
서,홍길서,서울시 중구
남,홍길남,서울시 강동구
북,홍길북,서울시 관악구
중,홍길중,부산시 중구


In [74]:
member.loc['동'] # 특정행이나 특정열만 access => 결과가 시리즈 #결과가 1차원이면 : 시리즈

Name                       홍길동
Age                         20
Email            hong@hong.com
Address                서울시 강동구
Birth      1999-01-01 00:00:00
Name: 동, dtype: object

In [76]:
member.loc['동':'동'] # 슬라이싱의 경우 결과가 2차원 => 데이터프레임

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01


## 4.3 iloc을 이용한 조회
- member.iloc[행번호, 열번호]
    * 행번호와 열번호 자리에 list ex.[0,2]
    * 행번호와 열이름 자리에 슬라이싱 from:to:by : from부터 by씩 증감하면서 to앞까지(to는 미포함)
    * ,열번호 생략시 모든 열

In [78]:
# 0번째~2번째 행 모든 열
# member.loc['동':'남']
member.iloc[0:3] # 열번호 생략시 모든 열

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01


In [83]:
member.iloc[0] # 결과가 1차원이면 시리즈, 데이터프레임으로 하고자 하면 슬라이싱 이용하거나,
               # pd.DataFrame()함수 이용

Name                       홍길동
Age                         20
Email            hong@hong.com
Address                서울시 강동구
Birth      1999-01-01 00:00:00
Name: 동, dtype: object

In [84]:
# 짝수번째 행의 맨 마지막 열을 제외한 데이터 추출
member.iloc[::2, :-1]

,Name,Age,Email,Address
동,홍길동,20,hong@hong.com,서울시 강동구
남,홍길남,21,nam@hong.com,서울시 강동구
중,홍길중,23,mid@hong.com,부산시 중구


In [85]:
# 모든 행의 마지막 열만 데이터프레임으로 추출
member.iloc[:, -1]

동   1999-01-01
서   1999-12-01
남   1999-11-01
북   1999-05-01
중   1999-01-30
Name: Birth, dtype: datetime64[ns]

In [86]:
# 0번째, 3번째행, 0번째, 3번째, 4번째 열
member.iloc[0:4:3, [0,3,4]]

,Name,Address,Birth
동,홍길동,서울시 강동구,1999-01-01
북,홍길북,서울시 관악구,1999-05-01


## 4.3 조건으로 조회
- df[조건] : 조건에 맞는 행(모든 열)
- df.loc[조건, 열이름] 또는 df[조건][열이름] : 조건에 맞는 행의 특정 열

In [89]:
member

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [90]:
member.Age > 22

동    False
서     True
남    False
북    False
중     True
Name: Age, dtype: bool

In [93]:
# Age가 22보다 큰 데이터 셋을 가져오기
member[member.Age>22]

,Name,Age,Email,Address,Birth
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [95]:
# Age가 22보다 큰 데이터의 Name과 Age
member[member.Age>22][['Name','Age']] #member[member.Age>22]['Name','Age'] 불가능

,Name,Age
서,홍길서,30
중,홍길중,23


In [98]:
# Age가 22보다 큰 데이터의 Name과 Age(loc사용)
member.loc[member.Age>22, ['Name','Age']]

,Name,Age
서,홍길서,30
중,홍길중,23


In [105]:
address = '서울시 강동구'
address2 = '부산시 중구'
print(address.startswith('서울시'))
print(address2.startswith('서울시'))
print(address.find('강동구')!=-1)
print(address2.count('강동구')>0)

True
False
True
False


※ 교안 pdf p.29 참조. 시리즈에 문자함수를 쓰기 위해 참조
https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

In [109]:
member.Address.str.startswith('서울시')

동     True
서     True
남     True
북     True
중    False
Name: Address, dtype: bool

In [110]:
# Address가 서울시로 시작하는 행들을 출력
member[member.Address.str.startswith('서울시')]

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01


In [111]:
# Address가 강동구가 포함된 행을 출력
member[member.Address.str.count('강동구')>0] 
# str.count('강동구') : 각 행의 Address 열에 '강동구'가 몇 번 등장하는지 세어줌
# 그 결과가 0보다 큰 것 → '강동구'가 포함된 행만 남김

member[member.Address.str.find('강동구')!=-1]
#str.find('강동구') :
#문자열 안에서 '강동구'를 찾은 위치를 반환 찾지 못하면 -1을 반환하기 때문에, != -1이면 '강동구'가 포함되어 있다는 뜻

member[member.Address.str.contains('강동구')]
#str.contains('강동구') : 해당 문자열이 포함되어 있으면 True, 없으면 False 반환 아주 직관적이고 가장 많이 쓰는 방식

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01


In [112]:
# Address에 '강동구'가 포함된 'Name', 'Age'열 조회

member[member.Address.str.contains('강동구')][['Name','Age']]
# 첫 번째 괄호: member.Address에 '강동구'가 포함된 행 필터링
# 두 번째 대괄호: 거기서 'Name', 'Age' 열만 선택

member.loc[member.Address.str.contains('강동구'), 'Name':'Age']
# loc[행, 열] 형식
# 행 부분: Address에 '강동구'가 포함된 조건
# 열 부분: 'Name'부터 'Age'까지 슬라이싱 (열 범위 지정)

,Name,Age
동,홍길동,20
남,홍길남,21


In [113]:
# 1999년도 태어난 데이터
member[member.Birth.dt.year==1999]

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


※ 데이터 프레임의 부분 데이터 조회 방법
- df[열이름] - 특정 열 모든 행조회
- df[조건] - 조건에 맞는 모든 행 조회
- df.loc[행이름, 열이름] : 이름조회, 조건(행이름 자리)으로 조회 가능
    * 열이름을 생략하면 모든 열
    * 행이름, 열이름 자리에서 list, 슬라이싱
- df.iloc[행번호, 열번호] : 번호 조회
    * 열번호를 생략하면 모든 열
    * 행번호, 열번호 자리에 list, 슬라이싱

In [114]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa


In [120]:
# 1. sepal_length 열만 출력
iris['sepal_length']

0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: sepal_length, Length: 150, dtype: float64

In [122]:
# 2. 0~10행까지 마지막 열을 제외한 데이터 (loc, iloc)
iris.loc[0:10, 'sepal_length':'petal_width']

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [155]:
# 3. 3~10 행중에 'sepal_length'와 'petal_length' 열만 (loc, iloc)
iris.loc[3:10, ['sepal_length','petal_length']]

,sepal_length,petal_length
3,4.6,1.5
4,5.0,1.4
5,5.4,1.7
6,4.6,1.4
7,5.0,1.5
8,4.4,1.4
9,4.9,1.5
10,5.4,1.5


In [157]:
# 4. 0번째, 50번째, 100번째 행의 모든 열 (loc, iloc)
iris.loc[[0, 50, 100]]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [161]:
# 5. 0번째, 25, 50번째, 75번, 100번째, 125번째 행의 petal_length와 petal_width (loc, iloc)
iris.loc[[0, 25, 50, 75, 100, 125], 'petal_length':'petal_width']

,petal_length,petal_width
0,1.4,0.2
25,1.6,0.2
50,4.7,1.4
75,4.4,1.4
100,6.0,2.5
125,6.0,1.8


In [145]:
# 6. species가 versicolor인 데이터의 모든 열
iris[iris.species.str.contains('versicolor')]

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [149]:
# 7. species가 setosa인 데이터 최초 5개 행만 출력
iris[iris.species.str.contains('setosa')][:5]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [162]:
# 7. 강사님 답안
iris[iris.species=='setosa'].head()
iris[iris.species=='setosa'].iloc[:5]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [152]:
# 8. sepal_length가 6.5이상인 데이터 최초 5개 행만 출력
iris[iris.sepal_length>6.5][:5]

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
52,6.9,3.1,4.9,1.5,versicolor
58,6.6,2.9,4.6,1.3,versicolor
65,6.7,3.1,4.4,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor


In [163]:
# 8. 강사님 답안
iris[iris.sepal_length>=6.5].head()
iris[iris.sepal_length>=6.5].iloc[:5]

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
52,6.9,3.1,4.9,1.5,versicolor
54,6.5,2.8,4.6,1.5,versicolor
58,6.6,2.9,4.6,1.3,versicolor
65,6.7,3.1,4.4,1.4,versicolor


In [154]:
# 9. sepal_length가 7.2이상인 데이터의 'sepal_length'와 'sepal_width'와 'petal_length'
iris[iris.sepal_length>7.2][['sepal_length', 'sepal_width', 'petal_length']]

,sepal_length,sepal_width,petal_length
105,7.6,3.0,6.6
107,7.3,2.9,6.3
117,7.7,3.8,6.7
118,7.7,2.6,6.9
122,7.7,2.8,6.7
130,7.4,2.8,6.1
131,7.9,3.8,6.4
135,7.7,3.0,6.1


In [164]:
# 9. 강사님 답안
iris.loc[iris.sepal_length >= 7.2, 'sepal_length':'petal_length']
iris[iris.sepal_length >= 7.2][['sepal_length','sepal_width','petal_length']]

,sepal_length,sepal_width,petal_length
105,7.6,3.0,6.6
107,7.3,2.9,6.3
109,7.2,3.6,6.1
117,7.7,3.8,6.7
118,7.7,2.6,6.9
122,7.7,2.8,6.7
125,7.2,3.2,6.0
129,7.2,3.0,5.8
130,7.4,2.8,6.1
131,7.9,3.8,6.4


In [167]:
# 10. versicolor종중에서 sepal_length가 6.5보다 큰 데이터의 모든 열(나중에 풀어볼것!)

In [165]:
# 10. 강사님 답안
iris[ (iris.species=='versicolor') & (iris.sepal_length>6.5)]

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
52,6.9,3.1,4.9,1.5,versicolor
58,6.6,2.9,4.6,1.3,versicolor
65,6.7,3.1,4.4,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
76,6.8,2.8,4.8,1.4,versicolor
77,6.7,3.0,5.0,1.7,versicolor
86,6.7,3.1,4.7,1.5,versicolor


In [168]:
# 10. 강사님 답안 +@
import numpy as np
np.logical_and(True, True)

True

In [169]:
iris[ np.logical_and(iris.species=='versicolor',
                   iris.sepal_length>6.5) ]

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
52,6.9,3.1,4.9,1.5,versicolor
58,6.6,2.9,4.6,1.3,versicolor
65,6.7,3.1,4.4,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
76,6.8,2.8,4.8,1.4,versicolor
77,6.7,3.0,5.0,1.7,versicolor
86,6.7,3.1,4.7,1.5,versicolor


---------------------------------- 4절 완료 ----------------------------------

# 5절. 데이터 추가 및 삭제
## 5.1 데이터프레임의 요소 삭제
- df.drop(labal, axis) : axis=0 :행삭제 / axis=1 : 열삭제           
- *labal = 행이름이나 열이름


In [172]:
member #컴퓨터를 재부팅한 경우 파일불러오는 함수를 통해 파일을 불러올것.
# member = pd.read_csv('data/ch11_member.csv', comment='#')
# member.index = ['동', '서', '남', '북', '중']
# member

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [180]:
# (1) 단일행 삭제
member.drop('중', ) #기본값 : 비울경우 axis값은 0이 됨. 즉, 행이 삭제됨

KeyError: "['중'] not found in axis"

In [175]:
# '동' 행이 없으면 에러가 발생함
member.drop('동', inplace=True)

In [176]:
# (2) 복수행 삭제
member.drop(['서','남']) # 복수삭제하고자 하는 요소를 리스트로 만듬.

,Name,Age,Email,Address,Birth
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01


In [177]:
# (3) 열 삭제
member.drop(labels=['Age','Birth'], axis=1)

,Name,Email,Address
서,홍길서,seo@hong.com,서울시 중구
남,홍길남,nam@hong.com,서울시 강동구
북,홍길북,buk@hong.com,서울시 관악구


## 5.2 데이터 프레임의 요소 추가

In [188]:
# 데이터 프레임에 열 추가하는 방법 1 : 일괄적인 데이터 장입
member['favarite_no'] = 7 # int 64
member

,Name,Age,Email,Address,Birth,favarite_no,fn
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,7,7.0
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,7,7.0
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,7,NaN


In [186]:
# 데이터 프레임에 열 추가하는 방법 2 : 리스트로 추가
member['fn'] = [7,7, None] #결측치 : None, np.nan, math.nan
member

,Name,Age,Email,Address,Birth,favarite_no,fn
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,7,7.0
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,7,7.0
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,7,NaN


In [187]:
# 결측치는 정수형 변환 불가(결측치 대체 후 형변환)
# member['fn'].astype('int')

In [193]:
# 데이터 프레임에 열 추가하는 방법 3 : 시리즈로 추가(데이터 갯수가 부족하면 나머지는 결측치로 들어감)
member = pd.read_csv('data/ch11_member.csv', comment='#')
member['fn2'] = pd.Series([7,7,7]) #1차원 데이터 생성
member

,Name,Age,Email,Address,Birth,fn2
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,7.0
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,7.0
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,7.0
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,NaN
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,NaN


In [196]:
member['fn'] = pd.Series(['test','test1','test2'], index=[0,3,4])
member
#member['fn'] =	데이터프레임에 'fn'이라는 새 열을 추가합니다
#pd.Series([...], index=[...])	3개의 문자열 데이터를 시리즈로 만들고, 어느 행(인덱스 번호)에 넣을지 지정합니다
#인덱스 0	'test'가 들어감
#인덱스 3	'test1'이 들어감
#인덱스 4	'test2'가 들어감
#인덱스 1, 2	시리즈에 값이 없으므로 NaN (결측치) 처리됨

,Name,Age,Email,Address,Birth,fn2,fn
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,7.0,test
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,7.0,NaN
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,7.0,NaN
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,NaN,test1
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,NaN,test2


In [199]:
# 행 추가 : 추가할 데이터를 데이터프레임 -> 기존 DataFrame, 추가할 DataFrame 연결
member = pd.read_csv('data/ch11_member.csv')
new_member = pd.DataFrame([{
    'Name'    :'홀길숙',
    'Age'     : 30,
    'Email'   : 'h@h.com',
    'Address' : '설',
    'Birth'   : '2010-01-01'
}])
new_member

,Name,Age,Email,Address,Birth
0,홀길숙,30,h@h.com,설,2010-01-01


In [204]:
# 행 추가시 member과 new_member 연결
member= pd.concat([member, new_member] ) 
#new_member를 아래쪽에 추가하는 경우, new_member를 윗쪽에 추가하고 싶다면 앞쪽에 작성
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30
5,#홍길뜽,24,non@not.com,없는구,2000-01-01
0,홀길숙,30,h@h.com,설,2010-01-01


In [205]:
pd.concat([member, new_member], axis=1) #axis=0 : 기본값 (행연결)
                                        #axis=1 : 열연결

,Name,Age,Email,Address,Birth,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,홀길숙,30.0,h@h.com,설,2010-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,NaN,NaN,NaN,NaN,NaN
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,NaN,NaN,NaN,NaN,NaN
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,NaN,NaN,NaN,NaN,NaN
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,NaN,NaN,NaN,NaN,NaN
5,#홍길뜽,24,non@not.com,없는구,2000-01-01,NaN,NaN,NaN,NaN,NaN
0,홀길숙,30,h@h.com,설,2010-01-01,홀길숙,30.0,h@h.com,설,2010-01-01


In [209]:
# 인덱스 재조정
# member.index = range(len(member)) -비추천되는 방식
member.reset_index(drop=True) # 기존의 index를 drop하고 새로운 연속된 index로 재조정
# member.reset_index() : 기존의 index를 컬럼에 편입시키고 새로운 연속된 index를 생성
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30
5,#홍길뜽,24,non@not.com,없는구,2000-01-01
6,홀길숙,30,h@h.com,설,2010-01-01


# 6절. 병합과 연결
## 6.1 merge()함수를 이용한 병합

In [215]:
df1 = pd.DataFrame({'key':['a','b','c','d'],
                   'c1':[1, 2, 3, 4]})
df2 = pd.DataFrame({'key':['a','b','c','e'],
                   'c2':[5, 6, 7, 8]})
display(df1) 
display(df2)

,key,c1
0,a,1
1,b,2
2,c,3
3,d,4


,key,c2
0,a,5
1,b,6
2,c,7
3,e,8


In [216]:
df1.merge(right=df2) # how='inner' : 양쪽 다 일치하는 데이터만 남김

,key,c1,c2
0,a,1,5
1,b,2,6
2,c,3,7


In [217]:
df1.merge(right=df2, how='left') # 왼쪽 데이터만 남기고 right에 매칭되는 것만 병합

,key,c1,c2
0,a,1,5.0
1,b,2,6.0
2,c,3,7.0
3,d,4,NaN


In [218]:
df1.merge(right=df2, how='right') # right 데이터만 남기고 왼쪽데이터에 매칭되는 것만 병합

,key,c1,c2
0,a,1.0,5
1,b,2.0,6
2,c,3.0,7
3,e,NaN,8


In [219]:
df1.merge(df2, how='outer') #양쪽 모두 다 남김

,key,c1,c2
0,a,1.0,5.0
1,b,2.0,6.0
2,c,3.0,7.0
3,d,4.0,NaN
4,e,NaN,8.0


In [220]:
# 공통된 속성이 없는 경우, ↑ 위에서의 예시는 공통된 속성이 존재했었음.
df3 = pd.DataFrame({'key3':['a','b','c','d'],
                   'c1':[1, 2, 3, 4]})
df4 = pd.DataFrame({'key4':['a','b','c','e'],
                   'c2':[5, 6, 7, 8]})
display(df3) 
display(df4)

,key3,c1
0,a,1
1,b,2
2,c,3
3,d,4


,key4,c2
0,a,5
1,b,6
2,c,7
3,e,8


In [221]:
df3.merge(right=df4, left_on='key3', right_on='key4', how='inner')

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,c,3,c,7


In [222]:
df3.merge(right=df4, left_on='key3', right_on='key4', how='outer')

,key3,c1,key4,c2
0,a,1.0,a,5.0
1,b,2.0,b,6.0
2,c,3.0,c,7.0
3,d,4.0,NaN,NaN
4,NaN,NaN,e,8.0


In [223]:
df3.merge(df4, left_index=True, right_index=True) # pd.concat([df3, df4],1)와 유사

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,c,3,c,7
3,d,4,e,8


## 6.2 concat()을 이용한 연결
- pd.concat( [df1, df2], axis )
    * axis=0(기본값) : 위아래로 연결
    * axis=1 : 좌우로 연결

In [226]:
df1 = pd.DataFrame({'key':['a','b','c','d'],
                   'c1':[1, 2, 3, 4]})
df2 = pd.DataFrame({'key':['a','b','c','e'],
                   'c1':[5, 6, 7, 8]})
pd.concat( [df1, df2], axis=0 ).reset_index(drop=True)

,key,c1
0,a,1
1,b,2
2,c,3
3,d,4
4,a,5
5,b,6
6,c,7
7,e,8


In [227]:
# 좌우 연결
df3 = pd.DataFrame({'key3':['a','b','c','d'],
                   'c1':[1, 2, 3, 4]})
df4 = pd.DataFrame({'key4':['a','b','c','e'],
                   'c2':[5, 6, 7, 8]})
pd.concat( [df3, df4], axis=1 )

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,c,3,c,7
3,d,4,e,8


# 7절 정렬(행이름, 열이름, 값에 의한 정렬)
'100' < '9'
- df.sort_index(axis) : 행(axis=0) 또는 열(axis=1) 이름으로 정렬
- df.sort_values(by=정렬기준이 될 열 이름, acending=True 또는 False, inplace=True 또는 False) : 값에 의한 정렬

In [229]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member.index = ['동','서','남','북','중']
member

,Name,Age,Email,Address,Birth
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 7.1 행 이름으로 정렬

In [232]:
member.sort_index(axis=0)
member.sort_index(axis='rows', inplace=True) # ascending=True 기본값(오름차순)
member

,Name,Age,Email,Address,Birth
남,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
동,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
북,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
서,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
중,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 7.2 열 이름으로 정렬

In [233]:
member.sort_index(axis=1)
member.sort_index(axis='columns', inplace=True) #ascending=True 기본값
member

,Address,Age,Birth,Email,Name
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중


## 7.3 값에 의한 정렬

In [234]:
# 만약 나이나 값 등이 동일한 경우에 대비하여 기준을 여러개 선정할 수 있다.
member

,Address,Age,Birth,Email,Name
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중


In [237]:
member.sort_values(by='Age') # 'Age' 열 데이터 기준으로 오름차순 정렬

,Address,Age,Birth,Email,Name
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서


In [239]:
member.sort_values(by='Age', ascending=False, inplace=True) #내림차순 정렬
member

,Address,Age,Birth,Email,Name
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동


In [242]:
# 'Address'기준으로 오름차순 정렬, 'Address'가 같으면 'Age'
member.sort_values(by=['Address','Age'])

,Address,Age,Birth,Email,Name
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서


In [244]:
# 'Address'기준으로 오름차순 정렬, 'Address'가 같으면 'Age' 내림차순
member.sort_values(by=['Address','Age'],
                  ascending=[True, False],
                  inplace=True)
member

,Address,Age,Birth,Email,Name
중,부산시 중구,23,1999-01-30,mid@hong.com,홍길중
남,서울시 강동구,21,1999-11-01,nam@hong.com,홍길남
동,서울시 강동구,20,1999-01-01,hong@hong.com,홍길동
북,서울시 관악구,22,1999-05-01,buk@hong.com,홍길북
서,서울시 중구,30,1999-12-01,seo@hong.com,홍길서


In [245]:
# iris 데이터 셋을 기준으로 정렬해볼것!
# (1)sepal_length값 기준 (sepal_length 같으면 sepal_width기준) 내림차순 정렬 적용(iris에)
# (2)행(index)기준 기준 정렬을 적용(iris에 적용)
# (3)열(column)이름 기준 정렬한 내용을 출력(iris에 적용 x)

In [246]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa


In [249]:
# (1) 
iris.sort_values(by=['sepal_length', 'sepal_width'],
                    ascending=False, inplace=True)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
131,7.9,3.8,6.4,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica
135,7.7,3.0,6.1,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
118,7.7,2.6,6.9,2.3,virginica


In [250]:
# (2)
iris.sort_index(inplace=True)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [251]:
# (3)
iris.sort_index(axis=1)

,petal_length,petal_width,sepal_length,sepal_width,species
0,1.4,0.2,5.1,3.5,setosa
1,1.4,0.2,4.9,3.0,setosa
2,1.3,0.2,4.7,3.2,setosa
3,1.5,0.2,4.6,3.1,setosa
4,1.4,0.2,5.0,3.6,setosa
...,...,...,...,...,...
145,5.2,2.3,6.7,3.0,virginica
146,5.0,1.9,6.3,2.5,virginica
147,5.2,2.0,6.5,3.0,virginica
148,5.4,2.3,6.2,3.4,virginica
